In [128]:
%load_ext autoreload
%autoreload 2
import jieba
import jieba.posseg as pseg
import nltk
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import torch
import re
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
ner_pipeline = pipeline(Tasks.named_entity_recognition, 'damo/nlp_raner_named-entity-recognition_chinese-base-news')


2025-02-13 16:24:28,846 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


2025-02-13 16:24:31,123 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-02-13 16:24:31,539 - modelscope - INFO - initiate model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-13 16:24:31,540 - modelscope - INFO - initiate model from location C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news.
2025-02-13 16:24:31,548 - modelscope - INFO - initialize model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-13 16:24:34,378 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\modelscope\utils\checkpoint.py:550: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will

In [10]:
with open("../data/BLCU/top_similar.pickle", 'rb') as handle:
    top_similar = pickle.load(handle)
with open("../data/HSK/HSK_levels.pickle", 'rb') as handle:
    hsk_dict = pickle.load(handle)

In [38]:
sentence = "特朗普在其“真相社交”平台上发文称，他和俄罗斯总统“同意让我们各自的团队立即开始谈判”，并邀请对方访问各自的首都。"

In [123]:
a = [np.float32(1.2), np.float32(1.8)]
int(np.argmax(a))

1

In [120]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-large-zh-v1.5')
embed_orig = model.encode("特朗普在其“真相社交”平台上发文称")
embed_new = model.encode("“真相社交”平台上发文称")
np.float32(model.similarity(embed_orig, embed_new)[0][0])

np.float32(0.82137644)

In [39]:
### tokenize sentence:
tokens = jieba.lcut(sentence)
tokens = [token for token in tokens if not re.match(r'^\W+$', token)] # remove punctuation

### NER on each token:
ner_output = ner_pipeline(sentence)['output'] # run NER pipeline, generate tokens
tokens_ner = list(set([d['span'] for d in ner_output if len(d['span'])>1])) # only collect NER longer than 1 character
tokens_no_ner = list(set(tokens) - set(tokens_ner)) # get all tokens that are NOT named entities


c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\modeling_utils.py:1113: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [68]:
list(blcu.iloc[top_similar[2],0])

['是是',
 '是的',
 '是否',
 '否',
 '就是',
 '肯定',
 '真是',
 '是不是',
 '不是',
 '还是',
 '而是',
 '对',
 '吗',
 '算是',
 '当然']

In [108]:
similarity_dict = {}

for idx in range(len(top_similar)):
    base_word = blcu.iloc[idx,0]
    top_list = list(blcu.iloc[top_similar[idx],0])
    flag = list(pseg.cut(base_word))[0].flag
    idx_list = []

    for word in top_list:
        tagged_word = list(pseg.cut(word))[0]  # POS tagging
        if tagged_word.flag==flag: 
            idx_list.append(word)
    if idx%500==0:
        print(idx)
    similarity_dict[base_word] = idx_list

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500


In [111]:
with open("../data/BLCU/similarity_dict.pickle", 'wb') as handle:
    pickle.dump(similarity_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [110]:
similarity_dict

{'的': [],
 '了': ['了了', '了得', '了账', '了事'],
 '是': ['是是', '是的', '是否', '肯定', '算是'],
 '我': ['我们',
  '本人',
  '我俩',
  '我国',
  '我方',
  '吾',
  '自我',
  '我处',
  '自己',
  '俺',
  '他人',
  '我们俩',
  '他'],
 '在': ['在此', '在行'],
 '他': ['他人', '她', '他们', '它', '他们俩', '有人', '某人', '我', '某个', '谁'],
 '你': ['你们', '你们俩', '他', '我', '自己', '她', '某某', '某人'],
 '也': ['也曾', '亦', '也就是', '也似', '也许', '亦即', '却又', '皆'],
 '着': ['着法', '着忙'],
 '这': ['这个',
  '这里',
  '这儿',
  '这些',
  '这样',
  '这么',
  '这种',
  '那',
  '这次',
  '这边',
  '这等',
  '这时',
  '那个'],
 '人': ['人家', '人士', '人们', '人名', '人类', '人员', '人人', '人物', '人民', '人身', '老人', '人称'],
 '道': ['道道', '道法', '道号', '道门'],
 '就': ['就是', '就得', '就要', '就此', '就正', '早就', '就地', '也就是', '就手', '就在于'],
 '个': ['些个'],
 '不': ['不许'],
 '说': ['说出', '说道', '说话', '明说', '却说', '说说', '说辞', '说法', '说上', '演说', '述说', '诉说'],
 '有': ['有的', '有着', '拥有', '应有', '已有', '存在', '还有', '有一点'],
 '上': ['上上', '上面', '上方', '上边', '上回', '以上', '上部', '上端', '之上'],
 '地': [],
 '那': ['那个',
  '那儿',
  '那里',
  '那么',
  '那次',
  '那边',
  '那样',
  '那时',
 

In [37]:
blcu = pd.read_csv('../data/BLCU/literature_wordfreq.release_UTF-8.txt', header = None, sep="\t",)
blcu.rename(columns={0:"character", 1:"frequency"}, inplace=True)
# df.set_index("character", inplace=True)
blcu["frequency"] = blcu["frequency"].rank(pct=True)
blcu.head()

,character,frequency
0,的,1.000000
1,了,0.999997
2,是,0.999994
3,我,0.999992
4,在,0.999989


In [28]:
for word in df.index[0:10]:
    tagged_word = list(pseg.cut(word))[0]  # POS tagging
    print(f"{word}: {tagged_word.word, tagged_word.flag}") 

的: ('的', 'uj')
了: ('了', 'ul')
是: ('是', 'v')
我: ('我', 'r')
在: ('在', 'p')
他: ('他', 'r')
你: ('你', 'r')
也: ('也', 'd')
着: ('着', 'uz')
这: ('这', 'r')


In [24]:
idx = 15000
top_list = list(df.index[top_similar[idx]])
flag = list(pseg.cut(df.index[idx]))[0].flag
print(f"Word:{df.index[idx]} --- POS tag:{flag}")

for word in top_list:
    tagged_word = list(pseg.cut(word))[0]  # POS tagging
    if tagged_word.flag==flag: 
        print(f"{word}: {tagged_word.word, tagged_word.flag}") 

Word:大度 --- POS tag:n
大方: ('大方', 'n')
气度: ('气度', 'n')
态度: ('态度', 'n')
风度: ('风度', 'n')
大气: ('大气', 'n')
尺度: ('尺度', 'n')
厚道: ('厚道', 'n')


In [2]:
import utils.LS_funcs as LS_funcs

2025-02-13 16:26:43,581 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


2025-02-13 16:26:45,867 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-02-13 16:26:46,292 - modelscope - INFO - initiate model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-13 16:26:46,293 - modelscope - INFO - initiate model from location C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news.
2025-02-13 16:26:46,304 - modelscope - INFO - initialize model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-13 16:26:49,486 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\modelscope\utils\checkpoint.py:550: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will

In [7]:
sentence = "有机构指出，根据以往经验，对于中央经济工作会需要关注会议的主题词，这决定了来年宏观经济主线以及政策导向。"
LS_funcs.LS_pipeline(sentence)

c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\modeling_utils.py:1113: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Best index is 3


'有机构指出，根据以往经验，对于中央经济工作会需要关注会议的主题词，这决定了来年宏观经济主线以及决策指向。'

In [5]:
from transformers import BertTokenizer, BartForConditionalGeneration, Text2TextGenerationPipeline, AutoModelForSeq2SeqLM
tokenizer = BertTokenizer.from_pretrained("../data/models/batch_128_15625_500000", local_files_only=True)
BART = BartForConditionalGeneration.from_pretrained("../data/models/batch_128_15625_500000", local_files_only=True)

In [8]:
text2text_generator = Text2TextGenerationPipeline(BART, tokenizer)
output = text2text_generator(sentence, max_length=128, do_sample=False)[0]['generated_text'].replace(" ","")
print(output)

Device set to use cpu


有机构指出，根据以往经验，中央经济工作会议需要关注的主题[4]，决定了今年宏观经济主线和政策导向。
